## Création du fichier résultat 

L'objectif est d'obtenir deux tableaux (un par feuille) qui regroupent: 
1- une vision globale sur les Objectifs de Développement Durable, 
2- une fiche indicateur dimensionnée sur un filtre dynamique obligatoire.

Import des librairies

In [33]:
import pandas as pd
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import subprocess
import papermill as pm

print(openpyxl.__version__)

3.1.3


Vérification de l'existence du fichier Excel de résultat 

In [39]:
from openpyxl import Workbook

path_file = '../OUTPUT/TDB_ODD.xlsx'

# Si le fichier n'existe pas, on le crée grâce au script d'intégration: 
notebook_path = '../SCRIPTS/integration.ipynb'

if not os.path.exists(path_file):
    try:
        pm.execute_notebook(
            notebook_path,
            '../SCRIPTS/TDB_ODD.ipynb',
            kernel_name='python3'
        )
        print("Fichier créé grâce au notebook d'intégration.")
    except Exception as e:
        print("Erreur lors de l'exécution du notebook d'intégration.")
        print(e)
else:
    print("Fichier trouvé.")

Fichier trouvé.


## Tableau bilan 
Elements du tableau: 
    Comparer les quantités d'action, ou l'effort cumulé, sur chaque ODD et par type d'indicateur. 
    Identifier les sujets cibles principaux de développement durable. 
    Identifier les indicateurs les plus représentés (indifféremment de l'ODD concerné). 
    Identifier la répartition du public concerné (sex, âge).
    Identifier la répartition des actions par geolocalisation. 

In [50]:
# Chargement feuille bilan  
wb = load_workbook(path_file)

if 'BILAN' not in wb.sheetnames:
    wb.create_sheet('BILAN')
    wb.save(path_file)
    print("Feuille bilan créée.")
else: 
    print("Feuille trouvée.")

bilan = wb['BILAN']

Feuille trouvée.


In [57]:
# Liste de valeurs uniques: 
wb = load_workbook(path_file)
ws = wb['BILAN']
ws['A2'] = '=UNIQUE(DATA!B:B)'
wb.save(path_file)
wb.close()

In [ ]:
# Tableau dynamique sur le nombre d'occurrence par ODD
# formule Excel: =